In [29]:
import json
import os
import sys
import boto3
from utils import bedrock, print_ww
from utils.utils import extract_model_info

## Setting up Bedrock API.

In [27]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


### Let's take a look at which models we have access to.

In [30]:
all_models = boto3_bedrock.list_foundation_models()
print(f"extract_model_info(all_models): {extract_model_info(all_models)}")

extract_model_info(all_models): [('Titan Text Large', 'amazon.titan-tg1-large'), ('Titan Text Embeddings', 'amazon.titan-e1t-medium'), ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'), ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'), ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl'), ('Stable Diffusion XL', 'stability.stable-diffusion-xl-v0'), ('J2 Grande Instruct', 'ai21.j2-grande-instruct'), ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'), ('Jurassic-2 Mid', 'ai21.j2-mid'), ('Jurassic-2 Mid', 'ai21.j2-mid-v1'), ('Jurassic-2 Ultra', 'ai21.j2-ultra'), ('Jurassic-2 Ultra', 'ai21.j2-ultra-v1'), ('Claude Instant', 'anthropic.claude-instant-v1'), ('Claude', 'anthropic.claude-v1'), ('Claude V1 100k', 'anthropic.claude-v1-100k'), ('Claude', 'anthropic.claude-v2'), ('Claude V2 100k', 'anthropic.claude-v2-100k'), ('Command', 'cohere.command-text-v14')]


In [23]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [11]:
prompt_data = """
Human: Tell me everything you know about JSON schema.

Assistant:"""

In [12]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [13]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is a summary of what I know about JSON schema:

- JSON schema is a specification for defining the structure and validation constraints of JSON data. It provides a declarative way to specify requirements for JSON documents.

- The main purposes of JSON schema are to validate JSON data based on a predefined schema, provide auto-generated documentation, and assist in rapid development by providing auto-completion in code editors.

- A JSON schema is itself a JSON document that declares constraints on the structure of other JSON documents. It defines things like required properties, data types, string lengths, validation patterns, etc.

- Some of the basic keywords and concepts in JSON schema include:

- type - specifies the JSON data type such as string, number, object, array, etc.

- properties - defines the keys and value types for JSON objects

- required - specifies required properties within an object

- minimum/maximum - specifies the min and max value of a number

- minLength

In [14]:
prompt_data = """
Human: Can you explain to me your prompting structure?  I see the Human and Assistant designations.  What else can you tell me about how to talk to you?

Assistant:"""

In [15]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [16]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 I don't actually have a complex prompting structure. I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. The "Human" and "Assistant" labels are there to clarify who is speaking, but you can just speak to me conversationally.


In [17]:
prompt_data = """
Human: Your role is to form function arguments based on the specification provide in 'FUNCTIONS'.  Users will ask you a question about the weather, and you'll attempt to form the function call that we can pass along to a tool to determine the response. Do not try to answer questions about the weather directly, use the provided tools.  The tools are specified using a json schema format, which defines how the arguments to the function are to be formed.

FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

Examples
###
Human: 
What's the weather like in Glasgow Scotland?
Assistant:
'function_call': {'name': 'get_current_weather',
    'arguments': '{
        "location": "Glasgow, Scotland",
        "format": "celsius"
    }'
}

Human: 
What's the weather like in Raleigh, NC?
Assistant:
'function_call': {'name': 'get_current_weather',
    'arguments': '{
        "location": "Raleigh, North Carolina",
        "format": "fahrenheit"
    }'
}
###

Human:
What's the weather like in Washington, DC?

Assistant:
"""

In [18]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2" 
accept = "application/json"
contentType = "application/json"

In [24]:
response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

 Here is the function call to get the current weather in Washington, DC:

'function_call': {'name': 'get_current_weather', 
    'arguments': '{
        "location": "Washington, DC",  
        "format": "fahrenheit"
    }'
}


# And Now for the Main Event.

Here's a class that prompts a language model for function arguments for a linear regression, based on data that we load into the context from and external source.


In [31]:
from prompt_template import PromptTemplate, load_text_file

In [ ]:
    #: Test using linear_trend_request_prompt.txt
    linear_trend_prompt = load_text_file('linear_trend_request_prompt.txt')
    linear_trend_prompt_template = PromptTemplate(linear_trend_prompt)
    # print(f"linear_trend_prompt_template.template: {linear_trend_prompt_template.template}")
    linear_trend_filled_prompt = linear_trend_prompt_template.fill(functions="<< add your function definitions here >>", 
    user_data="Here's a story involving some interesting commentary around trends in a market, supported by quantitative data.")
    print(f"linear_trend_filled_prompt: {linear_trend_filled_prompt}")